In [1]:
from navec import Navec
import numpy as np
import pandas as pd

path = '../../../static/embeddings/navec_hudlit_v1_12B_500K_300d_100q.tar'
navec = Navec.load(path)

In [2]:
def SentenceMaximum(sentence_):
    sentence_ = sentence_.split(" ")
    max_embedding_ = np.array([-1.0 for _ in range(300)])
    for i in range(len(sentence_)):
        if sentence_[i] in navec:
            t1 = max_embedding_ > navec[sentence_[i]]
            t2 = max_embedding_ < navec[sentence_[i]]
            max_embedding_ = max_embedding_ * t1 + navec[sentence_[i]] * t2
    return max_embedding_

In [3]:
def SentenceAverage(sentence_):
    sentence_ = sentence_.split(" ")
    average_embedding_ = np.array([0.0 for _ in range(300)])
    for i in range(len(sentence_)):
        if sentence_[i] in navec:
            average_embedding_ += navec[sentence_[i]]
    average_embedding_ /= len(sentence_)
    return average_embedding_

In [4]:
train_data = pd.read_csv('../../../static/datasets/original/validation_data_labeled.csv', sep='\t')
sentences = train_data.sentence

embeddings = pd.DataFrame({i : [] for i in range(300)})

for i in range(len(sentences)):
    embedding_sen = SentenceAverage(sentences[i])
    embeddings.loc[len(embeddings.index)] = embedding_sen

In [5]:
train_data.drop(columns=['sentence', 'entity', 'entity_tag', 'entity_pos_start_rel', 'entity_pos_end_rel'], inplace=True)
train_data = pd.concat([embeddings, train_data], axis=1)
# train_data = train_data[train_data.label != 0]
# train_data.label = train_data.label.replace(-1, 0)

In [6]:
train_data.to_csv('../../../static/embeddings/multi_average_val.csv', index=False)